### Os, Sync, Env

In [1]:
import os
import dotenv
dotenv.load_dotenv(dotenv_path="../.env")
import nest_asyncio
nest_asyncio.apply()

### Azure Configuration

In [2]:
azure_api_key = os.environ["AZURE_OPENAI_API_KEY"]
azure_base_url = os.environ["AZURE_BASE_URL"]
# From model URL endpoint
azure_api_version = "2024-08-01-preview"

### LLMs

In [ ]:
from ragas.llms import LlamaIndexLLMWrapper
from llama_index.llms.azure_openai import AzureOpenAI
evaluator_llm = LlamaIndexLLMWrapper(AzureOpenAI(
    model="gpt-4o-mini",
    deployment_name="gpt-4o-mini",
    api_key=azure_api_key,
    azure_endpoint=azure_base_url,
    api_version=azure_api_version,
))

### Evaluation Dataset

In [ ]:
# GROQ
QROQ_API_KEY = os.environ['GROQ_API_KEY_1']
from llama_index.llms.groq import Groq
groq_llm = Groq(model="llama3-8b-8192", api_key = QROQ_API_KEY, set_run_config=None)
print("Groq: "+str(groq_llm.complete("Hi")))
# Cohere
CO_API_KEY = os.environ['COHERE_API_KEY']
from llama_index.llms.cohere import Cohere
cohere_llm = Cohere(model="command-light",api_key=CO_API_KEY)
print("Cohere: "+str(cohere_llm.complete("Hi")))
# Fast Embed
from llama_index.embeddings.fastembed import FastEmbedEmbedding
fast_embed = FastEmbedEmbedding(model_name="BAAI/bge-base-en-v1.5")

from llama_index.core.settings import Settings
Settings.llm = groq_llm
Settings.embed_model = fast_embed

from llama_index.core.storage.docstore import SimpleDocumentStore
docstore = SimpleDocumentStore.from_persist_dir(persist_dir="D:/Learning New/GenAI/Project_RAG/RAG/Data_cleaning/doc_store")
documents = list(docstore.docs.values())

from llama_index.core import Document, VectorStoreIndex
index = VectorStoreIndex.from_documents(
    # remember, you must pass a list of documents!
    documents, 
    embed_model=Settings.embed_model,
    show_progress=True)

from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.core import PromptTemplate
# from prompts import QUESTION_GEN_PROMPT

vector_retriever = index.as_retriever(similarity_top_k=10)

bm25_retriever = BM25Retriever.from_defaults(docstore=index.docstore, similarity_top_k=10)

# QUERY_GEN_PROMPT_TEMPLATE=PromptTemplate(QUESTION_GEN_PROMPT)

retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    similarity_top_k=10,
    num_queries=1,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    # query_gen_prompt=QUERY_GEN_PROMPT_TEMPLATE, 
)

from llama_index.core import get_response_synthesizer
from llama_index.core.postprocessor import SimilarityPostprocessor

similarity_processor = SimilarityPostprocessor(similarity_cutoff=0.025)

response_synthsizer = get_response_synthesizer(llm=Settings.llm)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthsizer,
    node_postprocessors=[similarity_processor],
)

In [ ]:
from ragas.metrics import Faithfulness, SemanticSimilarity, FactualCorrectness, LLMContextRecall
from ragas.integrations.llama_index import evaluate
from datasets import load_dataset

from ragas import EvaluationDataset
eval_dataset = EvaluationDataset.from_pandas(df_dropped)
evaluator_llm = generator_llm
evaluator_embeddings = generator_embeddings
metrics = [
    LLMContextRecall(llm=evaluator_llm)
]
results = evaluate(dataset=eval_dataset, metrics=metrics, batch_size=2, query_engine=query_engine)

In [ ]:
results.to_pandas()